# 🎯 Few-shot fungi classification
This notebook provides a simple baselines for the few-shot classification of fungi. Both baselines require you to pre-extracted features using any encoder, e.g., BioCLIP, and DINOv2. For the classification, we provide:
1. **Centroid-based classifier** computes the class prototype by averaging the features of the support set. The class prototype is then used to classify the query set.
2. **Nearest neighbour classifier** classifies the query set by finding the nearest neighbour in the support set.

## ⚙️ Experiment Setup
Install and import for the required libraries.
Setting up pathing etc.

We also set path to dataset (which are downloaded automatically).

In [1]:
###Install required libraries:
%pip install git+https://github.com/mlfoundations/open_clip.git
%pip install faiss-cpu -qq


  Cloning https://github.com/mlfoundations/open_clip.git to /private/var/folders/zp/1_3lm2n11bbghqvjq26n5swm0000gn/T/pip-req-build-1ns13z5x
  Running command git clone --filter=blob:none --quiet https://github.com/mlfoundations/open_clip.git /private/var/folders/zp/1_3lm2n11bbghqvjq26n5swm0000gn/T/pip-req-build-1ns13z5x
  Resolved https://github.com/mlfoundations/open_clip.git to commit 7260a46e7b4bcf518f5200fea06da5bc85aae025
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
import os
import json
import yaml
from pathlib import Path
from types import SimpleNamespace
import argparse

import numpy as np
import pandas as pd
import torch
import faiss

import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from torchvision import transforms as tfms
import torchvision.transforms as T
import open_clip

from typing import Sequence, Tuple, Any, Dict, List, Optional, Union
import importlib
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [3]:
%pwd

'/Users/lpx/VScodeProjects/AMLS2_root/final_project'

In [4]:
# path to fungitatsic dataset
data_path = 'data/fungi-clef-2025/'


## 🔬 1. Define BioCLIP feature extractor

### BioCLIP: A Deep Learning Model for Biological Image Feature Extraction

🔗 [BioCLIP](https://huggingface.co/imageomics/bioclip) is a deep learning model designed for extracting high-quality feature embeddings from biological images. It is built on the **Contrastive Language-Image Pretraining (CLIP) framework**, specifically tailored for biological applications. The model is hosted on the **Hugging Face Hub**, making it easily accessible for researchers and developers working with biological image data.

#### Purpose and Applications

BioCLIP is designed to bridge the gap between visual data and semantic understanding in biological research. Its main applications include:

- **Automated Species Identification**: Extracting meaningful features from images of fungi, plants, or animals to assist in species recognition.
- **Biodiversity Monitoring**: Enabling large-scale ecological studies by processing images from camera traps or field surveys.
- **Medical and Microscopy Analysis**: Supporting tasks like cell classification, pathology slide analysis, and disease detection.
- **Bioinformatics and AI-powered Conservation**: Facilitating the development of models that leverage image-based insights for conservation efforts.

#### Key Features

- **Pretrained on Biological Datasets**: Unlike general-purpose CLIP models, BioCLIP has been fine-tuned on biological image datasets, improving its performance in domain-specific tasks.
- **Feature Extraction for Downstream Tasks**: Generates robust, high-dimensional embeddings that can be used for classification, clustering, or retrieval.
- **L2-Normalized Embeddings**: Ensures that extracted feature vectors have unit length, making them suitable for similarity comparisons.
- **Seamless Integration with PyTorch**: Designed to work within PyTorch-based workflows for machine learning and deep learning applications.

#### Code and Implementation

The implementation of BioCLIP for feature extraction is available in the following repository:

This model can be integrated into Python workflows using `open_clip`, allowing users to load the model, preprocess images, and extract embeddings efficiently.

In [5]:
class BioCLIP(torch.nn.Module):

    def __init__(self, device):
        """
        Initialize the BioCLIP feature extractor.
        """
        super(BioCLIP, self).__init__()
        self.device = device
        self.model = None
        self.processor = None
        self.size = (224, 224)

    def load(self):
        """
        Load the BioCLIP model and its associated image processor.

        The model is loaded from the Hugging Face Hub and moved to the specified device.
        """
        self.model, _, self.processor = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip')
        self.model.to(self.device)
        print('BioClip model loaded:', self.model)
        print('BioClip processor loaded:', self.processor)

    def extract_features(self, image):
        """
        Extract normalized feature embeddings from a given image.

        Args:
            image (PIL.Image.Image): The input image from which to extract features.

        Returns:
            torch.Tensor: A normalized feature embedding vector for the input image.

        Raises:
            ValueError: If the model has not been loaded prior to calling this method.
        """
        if self.model is None:
            raise ValueError('Model not loaded')
        image = image.resize(self.size, Image.BICUBIC)
        image_tensor_proc = self.processor(image)[None]
        features = self.model.encode_image(image_tensor_proc.to(self.device))
        return self.normalize_embedding(features)

    @staticmethod
    def normalize_embedding(embs):
        """
        Normalize the embedding vectors to have unit length.

        Args:
            embs (torch.Tensor): The raw embedding vectors.

        Returns:
            torch.Tensor: L2-normalized embedding vectors.
        """
        return torch.nn.functional.normalize(embs.float(), dim=1, p=2)


## 📙 2. Define dataset class for FungiTastic

`FungiTastic` is a PyTorch dataset class designed for handling the **Few-Shot subset** of the Danish Fungi dataset, with support for **train, validation, and test splits**. The class allows for efficient data handling, including image loading, embedding management, and transformation application.

#### Key Features:
- **Dataset Initialization**: Loads metadata (e.g., file paths, class IDs) from CSV files. Supports splits (`train`, `val`, `test`) and applies optional image transformations.
- **Embedding Handling**: Adds precomputed embeddings to the dataset and allows retrieval of embeddings for specific classes.
- **Data Access**: Implements the `__getitem__()` method to fetch images, class IDs, and file paths. It also returns embeddings if available.
- **Class Mapping**: Maps class IDs to species names and allows retrieving the category of samples by ID.
- **Sample Visualization**: Provides the `show_sample()` method to display images along with their class labels.

#### Functions:
- **`add_embeddings(embeddings)`**: Merges new embeddings with the dataset.
- **`get_embeddings_for_class(id)`**: Retrieves embeddings for a given class ID.
- **`_get_df(data_path, split)`**: Loads the dataset metadata into a pandas DataFrame.
- **`__getitem__(idx)`**: Fetches a sample by index, including image, category ID, and embedding (if available).
- **`__len__()`**: Returns the total number of samples in the dataset.
- **`get_class_id(idx)`**: Retrieves the class ID for a specific sample.
- **`show_sample(idx)`**: Displays an image with its associated class label.


In [49]:
class FungiTastic(torch.nn.Module):
    """
    Dataset class for the FewShot subset of the Danish Fungi dataset (size 300, closed-set).

    This dataset loader supports training, validation, and testing splits, and provides
    convenient access to images, class IDs, and file paths. It also supports optional
    image transformations.
    """

    SPLIT2STR = {'train': 'Train', 'val': 'Val', 'test': 'Test'}

    def __init__(self, root: str, split: str = 'val', transform=None):
        """
        Initializes the FungiTastic dataset.

        Args:
            root (str): The root directory of the dataset.
            split (str, optional): The dataset split to use. Must be one of {'train', 'val', 'test'}.
                Defaults to 'val'.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        super().__init__()
        self.split = split
        self.transform = transform
        self.df = self._get_df(root, split)
        self.feature_df = self.df.copy()
        self.feature_selection = ['year', 'month', 'habitat', 'country_code', 'iucnRedListCategory', 'substrate', 'coorUncert', 'latitude', 'longitude', 'region', 'district', 'poisonous', 'elevation', 'landcover', 'biogeographicalRegion']


        assert "image_path" in self.df
        if self.split != 'test':
            assert "category_id" in self.df
            self.n_classes = len(self.df['category_id'].unique())
            self.category_id2label = {
                k: v[0] for k, v in self.df.groupby('category_id')['species'].unique().to_dict().items()
            }
            self.label2category_id = {
                v: k for k, v in self.category_id2label.items()
            }

    def add_embeddings(self, embeddings: pd.DataFrame):
        """
        Updates the dataset instance with new embeddings.

        Args:
            embeddings (pd.DataFrame): A DataFrame containing an 'embedding' column.
                                       It must align with `self.df` in terms of indexing.
        """
        assert isinstance(embeddings, pd.DataFrame), "Embeddings must be a pandas DataFrame."
        assert "embedding" in embeddings.columns, "Embeddings DataFrame must have an 'embedding' column."
        assert len(embeddings) == len(self.df), "Embeddings must match dataset length."

        self.df = pd.merge(self.df, embeddings, on="filename", how="inner")

    def get_embeddings_for_class(self, id):
        # return the embeddings for class class_idx
        class_idxs = self.df[self.df['category_id'] == id].index
        return self.df.iloc[class_idxs]['embedding']

    @staticmethod
    def _get_df(data_path: str, split: str) -> pd.DataFrame:
        """
        Loads the dataset metadata as a pandas DataFrame.

        Args:
            data_path (str): The root directory where the dataset is stored.
            split (str): The dataset split to load. Must be one of {'train', 'val', 'test'}.

        Returns:
            pd.DataFrame: A DataFrame containing metadata and file paths for the split.
        """
        df_path = os.path.join(
            data_path,
            "metadata",
            "FungiTastic-FewShot",
            f"FungiTastic-FewShot-{FungiTastic.SPLIT2STR[split]}.csv"
        )
        df = pd.read_csv(df_path)
        df["image_path"] = df.filename.apply(
            lambda x: os.path.join(data_path, "FungiTastic-FewShot", split, '300p', x)
        )
        return df

    def __getitem__(self, idx: int):
        """
        Retrieves a single data sample by index.

        Args:
            idx (int): Index of the sample to retrieve.
            ret_image (bool, optional): Whether to explicitly return the image. Defaults to False.

        Returns:
            tuple:
                - If embeddings exist: (image, embedding, category_id, file_path)
                - If no embeddings: (image, category_id, file_path) (original version)
        """
        file_path = self.df["image_path"].iloc[idx].replace('FungiTastic-FewShot', 'images/FungiTastic-FewShot')

        if self.split != 'test':
            category_id = self.df["category_id"].iloc[idx]
        else:
            category_id = None

        image = Image.open(file_path)

        if self.transform:
            image = self.transform(image)

        # Check if embeddings exist
        if "embedding" in self.df.columns:
            emb = torch.tensor(self.df.iloc[idx]['embedding'], dtype=torch.float32).squeeze()
        else:
            emb = None  # No embeddings available


        return image, category_id, file_path, emb


    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.df)

    def get_class_id(self, idx: int) -> int:
        """
        Returns the class ID of a specific sample.
        """
        return self.df["category_id"].iloc[idx]

    def show_sample(self, idx: int) -> None:
        """
        Displays a sample image along with its class name and index.
        """
        image, category_id, _, _ = self.__getitem__(idx)
        class_name = self.category_id2label[category_id]

        plt.imshow(image)
        plt.title(f"Class: {class_name}; id: {idx}")
        plt.axis('off')
        plt.show()

    def get_category_idxs(self, category_id: int) -> List[int]:
        """
        Retrieves all indexes for a given category ID.
        """
        return self.df[self.df.category_id == category_id].index.tolist()
    
    def preprocess_metadata(self):
        scaler = StandardScaler()

        # ======================
        # 1. process time feature
        # ======================
        if 'month' in self.feature_df.columns:
            # generate cyclical features for month
            self.feature_df['month_sin'] = np.sin(2 * np.pi * self.feature_df['month'] / 12)
            self.feature_df['month_cos'] = np.cos(2 * np.pi * self.feature_df['month'] / 12)
            # scale the month columns
            self.feature_df['month_sin'] = scaler.fit_transform(self.feature_df['month_sin'])
            self.feature_df['month_cos'] = scaler.fit_transform(self.feature_df['month_cos'])
        # sclae the year column
        self.feature_df['year'] = scaler.fit_transform(self.feature_df['year'])
        # replace month feature with cyclical features
        self.feature_selection.remove('month')
        self.feature_selection.extend(['month_sin', 'month_cos'])
            
        # ======================
        # 2. handle missing values    
        # ======================
        # define missing value columns
        
        for col in self.feature_selection:
            # check if the column has missing values
            if self.feature_df[col].isnull().any():
                if self.feature_df[col].type in ['float64', 'int64']:
                    # add missing indicator column
                    self.feature_df[f"{col}_missing"] = self.feature_df[col].isnull().astype(int)
                    # fill missing values with mean value of the same category in that column   
                    mean_val = self.feature_df.groupby('category_id')[col].transform('mean')
                    self.feature_df[col] = self.feature_df[col].fillna(mean_val)
                    # scale the column by standard scaler
                    self.feature_df[col] = scaler.fit_transform(self.feature_df[col])
                    # extend missing indicator column to feature selection
                    self.feature_selection.append(f"{col}_missing")
                else:
                    # fill missing values with 'unknown'
                    self.feature_df[f'{col}_missing'] = self.feature_df[col].isnull().astype(int)
                    self.feature_df[col] = self.feature_df[col].fillna('unknown').astype(object)
                    # extend missing indicator column to feature selection
                    self.feature_selection.append(f"{col}_missing")
        
        # ======================
        # 3. drop unused columns
        # ======================

        for col in self.feature_df.columns:
            if col not in self.feature_selection:
                self.feature_df = self.feature_df.drop(columns=[col])
        
        # ======================
        # 4. one-hot encoding for categorical columns
        # ======================
        # get categorical columns

        for col in self.feature_df.columns:
            if self.feature_df[col].dtype == 'object':
                self.feature_df = pd.concat([self.feature_df, pd.get_dummies(self.feature_df[col], prefix=col)], axis=1)
                self.feature_df = self.feature_df.drop(columns=[col])


In [7]:
### Load the datasets

train_dataset = FungiTastic(root=data_path, split='train', transform=None)
test_dataset = FungiTastic(root=data_path, split='test', transform=None)

train_dataset.df.head(5)

,eventDate,year,month,day,habitat,countryCode,scientificName,kingdom,phylum,class,...,region,district,filename,category_id,metaSubstrate,poisonous,elevation,landcover,biogeographicalRegion,image_path
0,2021-01-24,2021,1.0,24.0,Mixed woodland (with coniferous and deciduous ...,DK,Xylohypha ferruginosa (Corda) S.Hughes,Fungi,Ascomycota,Eurotiomycetes,...,Sjælland,Næstved,0-3052832307.JPG,2421,wood,0,0.0,16.0,continental,data/fungi-clef-2025/FungiTastic-FewShot/train...
1,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,0-3061954303.JPG,386,wood,0,0.0,17.0,continental,data/fungi-clef-2025/FungiTastic-FewShot/train...
2,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,1-3061954303.JPG,386,wood,0,0.0,17.0,continental,data/fungi-clef-2025/FungiTastic-FewShot/train...
3,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,2-3061954303.JPG,386,wood,0,0.0,17.0,continental,data/fungi-clef-2025/FungiTastic-FewShot/train...
4,2021-02-27,2021,2.0,27.0,garden,DK,"Comatricha alta Preuss, 1851",Protozoa,Mycetozoa,Myxomycetes,...,Hovedstaden,Gribskov,3-3061954303.JPG,386,wood,0,0.0,17.0,continental,data/fungi-clef-2025/FungiTastic-FewShot/train...


In [13]:
# function to see statistics of dataframes for post processing
def show_df(df):
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     display(df)
    
    # show statistics
    display(df.describe())
    display(df.info())


In [14]:
show_df(train_dataset.df)

,year,month,day,latitude,longitude,coorUncert,observationID,category_id,poisonous,elevation,landcover
count,7819.000000,7819.000000,7819.00000,7819.000000,7819.000000,7805.000000,7.819000e+03,7819.000000,7819.000000,7722.000000,7716.000000
mean,2017.179818,7.673104,15.75956,54.902924,11.655231,78.642152,2.712784e+09,1216.306561,0.007290,72.065754,11.714619
std,5.799599,2.904563,9.08400,8.023161,11.812552,771.469730,5.699759e+08,702.018977,0.085075,163.440745,5.251739
min,1863.000000,1.000000,1.00000,-41.467014,-125.136213,1.000000,2.237852e+09,0.000000,0.000000,0.000000,0.000000
25%,2016.000000,6.000000,8.00000,55.504575,9.966628,10.000000,2.238483e+09,609.000000,0.000000,0.000000,8.000000
50%,2019.000000,8.000000,16.00000,55.826593,11.664595,25.000000,2.449444e+09,1228.000000,0.000000,14.000000,12.000000
75%,2020.000000,10.000000,24.00000,56.168667,12.338963,50.000000,3.080243e+09,1827.000000,0.000000,68.000000,17.000000
max,2021.000000,12.000000,31.00000,78.190238,147.193711,50000.000000,4.847353e+09,2426.000000,1.000000,1600.000000,17.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7819 entries, 0 to 7818
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   eventDate              7819 non-null   object 
 1   year                   7819 non-null   int64  
 2   month                  7819 non-null   float64
 3   day                    7819 non-null   float64
 4   habitat                7819 non-null   object 
 5   countryCode            7819 non-null   object 
 6   scientificName         7819 non-null   object 
 7   kingdom                7819 non-null   object 
 8   phylum                 7819 non-null   object 
 9   class                  7779 non-null   object 
 10  order                  7713 non-null   object 
 11  family                 7643 non-null   object 
 12  genus                  7819 non-null   object 
 13  specificEpithet        7819 non-null   object 
 14  hasCoordinate          7819 non-null   bool   
 15  spec

None

## ⏳ 3. Precompute embeddings using BioCLIP

This code snippet demonstrates how to precompute image embeddings using the **BioCLIP** model. It loads the BioCLIP model, processes a given dataset, and stores the generated embeddings.

#### Steps:
1. **Initialize the Model**: The code sets up the **BioCLIP** model on the available device (GPU if available, else CPU). The model is then loaded and set to evaluation mode (`model.eval()`).
2. **Generate Embeddings**:
   - The `generate_embeddings()` function loops through all the samples in the dataset.
   - For each image, it extracts the feature embeddings using `model.extract_features()`.
   - The extracted embeddings are stored in a pandas DataFrame with the image filename.
3. **Add Embeddings to Dataset**: The generated embeddings are then added to the respective dataset (`test_dataset` and `train_dataset`) using the `add_embeddings()` method.

#### Key Components:
- **BioCLIP model**: Used to extract embeddings for each image in the dataset.
- **Data Handling**: `im_names` stores image filenames, and `embs` stores the corresponding embeddings.
- **Embedding Storage**: Embeddings are stored in a pandas DataFrame, which is later merged with the dataset.

This process enables efficient embedding-based learning and retrieval tasks on the dataset.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BioCLIP(device=device)
model.load()
model.eval()



open_clip_pytorch_model.bin:   0%|          | 0.00/599M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/469 [00:00<?, ?B/s]

BioClip model loaded: CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-

BioCLIP(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
      (patch_dropout): Identity()
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-11): 12 x ResidualAttentionBlock(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ls_1): Identity()
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): GELU(approximate='none')
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ls_2): Identity()
          )
        )
      

In [52]:
print(model.model)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [9]:
def generate_embeddings(dataset):

    idxs = np.arange(len(dataset))
    im_names, embs = [], []
    for idx in tqdm(idxs):
        im, label, file_path, _ = dataset[idx]

        with torch.no_grad():
            feat = model.extract_features(im)

        # feat_quant = model.quantize_normalized_embedding(feat)

        im_names.append(os.path.basename(file_path))
        embs.append(feat.detach().cpu().numpy())

    #print('feature shape:', feat.shape)
    # # feature shape: torch.Size([1, 512])
    embeddings = pd.DataFrame({'filename': im_names, 'embedding': embs})

    return embeddings

In [10]:
embeddings = generate_embeddings(test_dataset)
test_dataset.add_embeddings(embeddings)

100%|██████████| 1911/1911 [02:18<00:00, 13.84it/s]


In [16]:
embeddings = generate_embeddings(train_dataset)
train_dataset.add_embeddings(embeddings)

100%|██████████| 7819/7819 [10:22<00:00, 12.56it/s]


In [17]:
embeddings.head(5)

,filename,embedding
0,0-3052832307.JPG,"[[-0.06053972, 0.027627386, -0.07043048, 0.029..."
1,0-3061954303.JPG,"[[-0.00093170325, 0.030387087, -0.015782572, 0..."
2,1-3061954303.JPG,"[[-0.033966947, 0.006955753, -0.0068196007, -0..."
3,2-3061954303.JPG,"[[-0.0064156693, -0.014657668, -0.04538401, -0..."
4,3-3061954303.JPG,"[[0.008788146, -0.017472612, -0.0038803064, -0..."


In [ ]:
# save precomputed embeddings to disk using vector database Pinecone
from pinecone import Pinecone, ServerlessSpec

import openai  # Only if you're using OpenAI embeddings

# Initialize Pinecone
PINECONE_API_KEY = "pcsk_2rZBXu_TSTXNscAXq9G2PHudHdYmjDuBYKmLkfkVT73GemxBUvfjcSWuPvp3RKsuSvqM4T"

pc = Pinecone(api_key=PINECONE_API_KEY)

# Create or connect to index
index_name = "FungiTastic-Image-Embeddings(BioCLIP generated)"

# Create index if it doesn't exist
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=512, # Replace with your model dimensions
        metric="cosine", # Replace with your model metric
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )

# Connect to the index
index = pinecone.Index(index_name)

## 🏃🏻‍♂4. Run the few-shot classifier on the precomputed features

### 4.1 Classification with kNN

This code defines a **Nearest Neighbor (NN) Classifier** that uses precomputed embeddings to make predictions by comparing them to class prototypes. It employs **cosine similarity** to match test embeddings with training embeddings, making it suitable for few-shot and embedding-based classification tasks.

#### Key Steps:

1. **Classifier Initialization**:
   - The `NNClassifier` class is initialized with the training embeddings and sets up a similarity search index using **FAISS** (Facebook AI Similarity Search).
   - The `build_index()` method creates an index for efficient similarity searches based on the cosine similarity between embeddings.

2. **Making Predictions**:
   - The `make_prediction()` method computes the similarity between each test embedding and the stored class prototypes. It returns the predicted class labels (`cls`) and confidence scores (`conf`) based on the similarity values.
   - **FAISS** is used to search the nearest prototypes for each test embedding.

3. **Generating Predictions for the Test Dataset**:
   - After initializing the classifier, predictions are made for the **test dataset** using the `make_prediction()` method.
   - Predicted labels are added to the test dataset (`test_dataset.df["preds"]`).

4. **Submission Generation**:
   - The predictions are grouped by `observationID` and formatted as a space-separated string of class IDs.
   - The predictions are saved to a CSV file for submission.

#### Key Components:
- **FAISS**: Efficiently computes nearest neighbor search using cosine similarity.
- **Embedding-based Classification**: The model leverages precomputed embeddings for each sample and matches them to class prototypes.
- **Prediction Confidence**: The similarity values serve as confidence scores for each prediction.

This approach is effective for embedding-based classification tasks where models make predictions based on the similarity of embeddings to predefined class prototypes.


In [30]:
class NNClassifier():
    def __init__(self, train_embeddings, device='cuda'):
        """
        :param cfg: config object, namespace
        :param train_embeddings: list of C torch arrays of shape [N_C, D] where N_C is the number of training samples
        of class C and D is the dimensionality of the embeddings
        """

        self.device = device
        self.index, self.idx2cls = self.build_index(train_embeddings)

    def make_prediction(self, embeddings, plot_sim_hist=False, ret_probs=False):
        """
        :param embeddings: torch.Tensor of shape (batch_size, n_channels, height, width)
        :return: probabilities of shape (batch_size, n_classes) computed based on
        the similarity of the embeddings to the class prototypes
        """
        # compute the similarity of each embedding to each prototype
        # embeddings - [N, D], class_prototypes - [C, D]
        similarities, indices = self.index.search(embeddings, 10)
        # get the classes for the indices
        cls = self.idx2cls[indices.squeeze()]
        # get the confidence of the prediction
        conf = similarities

        return cls, conf

    def build_index(self, train_dataset):
        idx2cls = train_dataset.df.category_id.values
        # concatenate the embeddings
        embs = np.array(train_dataset.df.embedding.values.tolist(), dtype=np.float32).squeeze()
        # build the index for cosine similarity search
        index = faiss.IndexFlatIP(embs.shape[1])
        index.add(embs)
        return index, idx2cls

In [45]:
classifier = NNClassifier(train_dataset, device='cpu')

cls, conf = classifier.make_prediction(np.array(test_dataset.df.embedding.values.tolist(), dtype=np.float32).squeeze())

In [32]:
cls.shape

(1911, 10)

In [33]:
conf.shape

(1911, 10)

In [46]:
cls[0], conf[0]

(array([2120, 1654, 1157, 2307, 1502, 1269, 2352, 1478, 2252,  670]),
 array([0.48146302, 0.47505337, 0.47167265, 0.4712574 , 0.46617448,
        0.4562657 , 0.45316428, 0.4522664 , 0.45123884, 0.45115402],
       dtype=float32))

In [48]:
type(cls)

numpy.ndarray

In [50]:
# combine cls and conf in the way of (cls, similarity) pair
cls_conf = list(zip(cls, conf))
cls_conf[0]

(array([2120, 1654, 1157, 2307, 1502, 1269, 2352, 1478, 2252,  670]),
 array([0.48146302, 0.47505337, 0.47167265, 0.4712574 , 0.46617448,
        0.4562657 , 0.45316428, 0.4522664 , 0.45123884, 0.45115402],
       dtype=float32))

In [49]:
test_dataset.df.groupby("observationID")

In [ ]:
for idx in range(len(test_dataset.df)):

    test_dataset.df.iloc[idx, 'preds'] = cls[idx]

In [38]:
test_dataset.df["preds"] = cls
submission = (
    test_dataset.df.groupby("observationID")["preds"]
    .apply(lambda x: " ".join(map(str, sorted(set(x)))))  # Sort and remove duplicates
    .reset_index()
)

submission = submission.rename(columns={"observationID": "observationId", "preds": "predictions"})
submission = submission.drop_duplicates(subset="observationId")
submission.to_csv(data_path + "baseline-submission-with-nn.csv", index=None)

In [27]:
!pwd

/content


In [39]:
test_dataset.df.head(5)

,eventDate,year,month,day,habitat,countryCode,hasCoordinate,substrate,latitude,longitude,...,region,district,filename,metaSubstrate,elevation,landcover,biogeographicalRegion,image_path,embedding,preds
0,2023-01-08,2023.0,1.0,8.0,Deciduous woodland,DK,True,dead wood (including bark),55.993269,12.290225,...,Hovedstaden,Hillerød,0-4100099350.JPG,wood,43.0,12.0,continental,/content/drive/MyDrive/KaggleHub/fungi-clef-20...,"[[0.02542315, -0.0038490202, -0.03277941, 0.02...",2120
1,2023-01-10,2023.0,1.0,10.0,Unmanaged deciduous woodland,DK,True,dead wood (including bark),55.973423,10.550840,...,Midtjylland,Samsø,1-4100096393.JPG,wood,64.0,12.0,continental,/content/drive/MyDrive/KaggleHub/fungi-clef-20...,"[[-0.047158033, 0.013674775, -0.05174886, -0.0...",2383
2,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,...,Islas Canarias,Santa Cruz de Tenerife,0-4100103428.JPG,jord,0.0,0.0,NaN,/content/drive/MyDrive/KaggleHub/fungi-clef-20...,"[[0.016380193, -0.03546471, -0.013923304, -0.0...",1278
3,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,...,Islas Canarias,Santa Cruz de Tenerife,1-4100103428.JPG,jord,0.0,0.0,NaN,/content/drive/MyDrive/KaggleHub/fungi-clef-20...,"[[0.034515046, -0.008639909, 0.027649816, 0.01...",1278
4,2023-01-09,2023.0,1.0,9.0,lawn,ES,True,soil,28.501376,-17.874959,...,Islas Canarias,Santa Cruz de Tenerife,2-4100103428.JPG,jord,0.0,0.0,NaN,/content/drive/MyDrive/KaggleHub/fungi-clef-20...,"[[0.048006557, -0.051189955, 0.036810976, -0.0...",1278


### 4.2 Classification with Prototypes

The `PrototypeClassifier` is a PyTorch model designed for **embedding-based classification** using **prototype-based classification**. It learns the class prototypes by averaging the embeddings of each class and uses cosine similarity to classify new embeddings based on their proximity to the prototypes.

#### Key Steps:

1. **Initialization**:
   - The model is initialized with a **train dataset** and computes class prototypes using embeddings of each class.
   - The `class_prototypes` are stored as **non-trainable parameters** (`requires_grad=False`), representing the average embeddings of each class.

2. **Embedding Retrieval**:
   - The `_get_classifier_embeddings()` method retrieves the embeddings for each class in the training dataset.
   - If a class has no embeddings, a zero vector is used as a placeholder for that class.

3. **Prototype Computation**:
   - The `get_prototypes()` method computes the **prototype** for each class by averaging its embeddings. These prototypes serve as the central representation of each class.

4. **Prediction**:
   - The `make_prediction()` method calculates the cosine similarity between each input embedding and all class prototypes.
   - The predicted class (`cls`) is the one with the highest similarity, and the prediction confidence (`conf`) is derived from the maximum softmax probability over the similarities.

#### Key Components:
- **Class Prototypes**: Each class is represented by its average embedding, which acts as the class prototype.
- **Cosine Similarity**: The model uses cosine similarity to compare input embeddings with class prototypes and determine the predicted class.
- **Prediction Confidence**: The confidence is computed using the softmax of the cosine similarity values, providing a measure of certainty for each prediction.

This classifier is well-suited for tasks where class prototypes (average embeddings) can effectively represent the classes, and it works well with embedding-based models for few-shot or metric learning tasks.


In [ ]:
class PrototypeClassifier(torch.nn.Module):
    def __init__(self, train_dataset, device='cuda'):
        super().__init__()
        self.device = device
        self.train_dataset = train_dataset

        class_embeddings, _ = self._get_classifier_embeddings(train_dataset)
        self.class_prototypes = torch.nn.Parameter(self.get_prototypes(class_embeddings), requires_grad=False)

    def _get_classifier_embeddings(self, dataset_train):
        class_embeddings = []
        empty_classes = []
        n_classes = min(torch.inf, dataset_train.n_classes)
        for cls in range(n_classes):
            cls_embs = dataset_train.get_embeddings_for_class(cls)
            if len(cls_embs) == 0:
                # if no embeddings for class, use zeros
                empty_classes.append(cls)
                class_embeddings.append(torch.zeros(1, dataset_train.emb_dim))
            else:
                class_embeddings.append(torch.tensor(np.vstack(cls_embs.values)))
        return class_embeddings, empty_classes

    def get_prototypes(self, embeddings):
        return torch.stack([class_embs.mean(dim=0) for class_embs in embeddings])

    def make_prediction(self, embeddings):
        similarities = torch.nn.functional.cosine_similarity(embeddings, self.class_prototypes, dim=-1)
        cls = torch.argmax(similarities, dim=1)
        conf = torch.nn.functional.softmax(similarities, dim=1).max(dim=1).values
        return cls, conf

In [ ]:
classifier = PrototypeClassifier(train_dataset, device='cpu')
cls, conf = classifier.make_prediction(torch.tensor(np.array(test_dataset.df.embedding.values.tolist(), dtype=np.float32)))

In [ ]:
test_dataset.df["preds"] = cls
submission = (
    test_dataset.df.groupby("observationID")["preds"]
    .apply(lambda x: " ".join(map(str, sorted(set(x)))))  # Sort and remove duplicates
    .reset_index()
)

submission = submission.rename(columns={"observationID": "observationId", "preds": "predictions"})
submission = submission.drop_duplicates(subset="observationId")
submission.to_csv("baseline-submission-with-prototypes.csv", index=None)